In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, applications, optimizers, losses
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

### Load the data

In [ ]:
setattr(tfds.image_classification.cats_vs_dogs, '_URL',"https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")
(train_ds, test_ds), metadata = tfds.load('cats_vs_dogs', split=['train[:35%]', 'train[90%:95%]'],
                                          with_info=True, as_supervised=True, shuffle_files=True,)

In [ ]:
# info related to the dataset
metadata

### More about the data

In [ ]:
tot_train_images = train_ds.__len__().numpy()
tot_test_images = test_ds.__len__().numpy()

print(f"The total number of training images is: {tot_train_images}")
print(f"The total number of test images is: {tot_test_images}")
im_iter = train_ds.__iter__()

In [ ]:
# visualize an image in the training dataset
image, label = next(im_iter)
plt.figure(figsize=(8,6))
label = (1-label.numpy())*"Cat" + label.numpy()*"Dog"
plt.imshow(image.numpy())
plt.title(label)
print(f"The original shape of this image is: {image.numpy().shape}")

As you can see by running multiple times the code cell above, the images in the dataset have different shapes (i.e. different number of pixels)

We need to resize the images to a common shape

### Pre-process the data

In [ ]:
def image_formatting(image, label):
    # converting pixel values (uint8) to float32 type
    image = tf.cast(image, tf.float32)
    # normalizing the data to be in range of -1, +1
    image = applications.resnet_v2.preprocess_input(image)
    # resizing all images to a shape of 224x*224*3
    image = tf.image.resize(image, (224, 224))
    return image, label
    
# converting all images to same shape and formatting them for quicker training of the model
train = train_ds.map(image_formatting)
test = test_ds.map(image_formatting)

In [ ]:
# suffle the data abd create batches
batch_size = 128
shuffle_size = 1000

train_batches = train.shuffle(shuffle_size).batch(batch_size)
test_batches = test.batch(batch_size)

### Create a pre-trained model

We will exploit keras.applications API to create a pre-trained model from ResNet152V2. This is a very deep convolutional neural network with ~60 million parameters that was trained using the ImageNet dataset. We will not re-train the entire network on the Cats_vs_Dog data (This would be computationally demanding). Instead, we will keep the majority of the network by replacing only the last fully connected layers with new layers that we are going to train on our data. Basically, ResNet152V2 will be used for feature extraction.

keras.applications: https://keras.io/api/applications/
ResNet152V2: https://keras.io/api/applications/resnet/#resnet152v2-function

In [ ]:
# create the base model based on ResNet152V2
base_model = applications.ResNet152V2(
    weights='imagenet',  # Load weights pre-trained on the ImageNet dataset
    input_shape=(224, 224, 3),
    include_top=False) # do not include the classifier at the top

In [ ]:
model = models.Sequential()

## TODO
# add the entire base_model as "first layer"
model.add(...) 

# add a GlobalAveragePooling2D layer
model.add(...)

# add to the model a Dense layer with 256 neurons and ReLu activation
model.add(...)

# add to the model a Dense layer with 1 neurons and Sigmoid activation (Why??)
model.add(...)

# do not train the first layer (ResNet/base_model) of the model as it is already trained
model.layers[0].trainable = False

In [ ]:
model.summary()

In [ ]:
## TODO
# compile the model
model.compile(...)

In [ ]:
## TODO
# train the model for 5 epochs: use a batch size of 128 data samples
model.fit(..., workers=32)